In [1]:
import math
import numpy as np
import tensorflow as tf

import evaluate

In [2]:
def conv_block(x, n_filters):
    x = tf.keras.layers.Conv1D(n_filters, 3, padding='same', activation='relu')(x)
    return x

def downsample_block(x, pool_size=4):
    x = tf.keras.layers.MaxPool1D(pool_size)(x)
    return x

def get_model_CMV1(n_filters=64):
    # CMNet V1
    model_input = tf.keras.layers.Input(shape=(4000,2))

    x = model_input
    for i in range(5):
        x = conv_block(x, n_filters)
        x = downsample_block(x)

    x = tf.keras.layers.Conv1D(1, 3, padding='valid', activation='linear')(x)

    model = tf.keras.Model(model_input, x)
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.MeanSquaredError(),
                  metrics=tf.keras.metrics.RootMeanSquaredError())
    return model

'''
print('CMV1-XS')
get_model_CMV1(n_filters=16).summary()
print('CMV1-S')
get_model_CMV1(n_filters=32).summary()
print('CMV1-L')
get_model_CMV1(n_filters=192).summary()
'''
print('CMV1')
get_model_CMV1().summary()

CMV1
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4000, 2)]         0         
_________________________________________________________________
conv1d (Conv1D)              (None, 4000, 64)          448       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 1000, 64)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1000, 64)          12352     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 250, 64)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 250, 64)           12352     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 62, 64)            0

In [3]:
for model in [get_model_CMV1(n_filters=16), get_model_CMV1(n_filters=32), get_model_CMV1(), get_model_CMV1(n_filters=192)]:
    Wsave = model.get_weights()
    for dataset in ['BIDMC32HR', 'BIDMC32RR', 'BIDMC32SpO2']:
        mse_list = []
        rmse_list = []

        train_x = np.load(f'dataset/{dataset}_train_x.npy')
        train_y = np.load(f'dataset/{dataset}_train_y.npy')
        test_x  = np.load(f'dataset/{dataset}_test_x.npy')
        test_y  = np.load(f'dataset/{dataset}_test_y.npy')
        train_x = np.swapaxes(train_x, 1, 2)
        test_x = np.swapaxes(test_x, 1, 2)
        
        for i in range(3):
            model.set_weights(Wsave)

            # model selection
            # model = get_model_CMV1()
            model.fit(x=train_x, y=train_y, batch_size=32, epochs=128, verbose=0)
            mse, rmse = model.evaluate(test_x, test_y)
            mse_list.append(mse)
            rmse_list.append(rmse)

        print(dataset)
        print('MEAN')
        print('MSE :', np.mean(mse_list))
        print('RMSE:', np.mean(rmse_list))

75/75 [==============================] - 0s 5ms/step - loss: 82.3451 - root_mean_squared_error: 9.0744
BIDMC32HR
MEAN
MSE : 130.17672475179037
RMSE: 11.298983891805014
75/75 [==============================] - 0s 5ms/step - loss: 30.6841 - root_mean_squared_error: 5.5393
BIDMC32RR
MEAN
MSE : 30.51386324564616
RMSE: 5.513475259145101
75/75 [==============================] - 0s 5ms/step - loss: 30.0072 - root_mean_squared_error: 5.4779
BIDMC32SpO2
MEAN
MSE : 33.053059895833336
RMSE: 5.741360187530518
75/75 [==============================] - 0s 5ms/step - loss: 215.9751 - root_mean_squared_error: 14.6961
BIDMC32HR
MEAN
MSE : 209.4052480061849
RMSE: 14.469854354858398
75/75 [==============================] - 0s 5ms/step - loss: 28.3758 - root_mean_squared_error: 5.3269
BIDMC32RR
MEAN
MSE : 26.27297846476237
RMSE: 5.123415946960449
75/75 [==============================] - 0s 5ms/step - loss: 26.3278 - root_mean_squared_error: 5.1311
BIDMC32SpO2
MEAN
MSE : 25.602357228597004
RMSE: 5.058067321